<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=77424fa73fb0855b83b11ef85b65c090e0b53241dd1d681761ce1549b6a1adf5
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0


    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-11 09:24:41
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  11.04 L
Current:  1.42 C
-------------------
Today PnL: 25.48 K (0.18%)
Current PnL: -17.66 L (-11.88%)
CY Booked + Current PnL: -6.63 L (-4.46%)
-------------------
Total profit:  2.08 L
Total loss:  -19.74 L
-------------------
Total Booked + Current PnL: 20.35 L (16.72%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.79%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 77.31 L (54.62%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.46%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-12.98,62.0,H-LC,11.22,173987.0,17810.0,8282.0,-0.26,11.40,4.76,16.71,22.0,2.15,1.33,30.83,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-11.34,61.0,M-LC,4.29,218294.0,16814.0,10391.0,0.68,8.35,4.76,13.51,60.0,1.62,1.67,21.99,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,106.19,63.0,M-SC,5.26,88818.0,-11959.0,12053.0,0.21,-11.87,13.57,0.09,245.0,-0.99,0.68,16.69,OX40N,NTT,DURABLES
41,ITC,452.00,-37.08,56.0,H-LC,3.02,202690.0,2552.0,18343.0,0.35,1.28,9.05,10.44,4.0,0.14,1.55,6.91,X40,NTT,FMCG
49,NATIONALUM,247.44,-43.66,68.0,H-MC,3.38,111761.0,11257.0,19379.0,0.83,11.20,17.34,30.49,79.0,0.58,0.86,47.24,MH,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,41.94,62.0,M-MC,6.97,224377.0,-585.0,168238.0,0.45,-0.26,74.98,74.53,192.0,-0.00,1.72,31.91,XY24,BTT,STEEL
4,ANGELONE,3033.00,8.16,40.0,H-SC,6.40,191601.0,595.0,57097.0,0.03,0.31,29.80,30.21,157.0,0.01,1.47,21.26,X40N,NTT,FINANCE
54,RAJOOENG,143.33,-32.03,49.0,H-SC,10.60,103100.0,600.0,40230.0,3.87,0.59,39.02,39.83,114.0,0.01,0.79,7.77,AR,ATH,MISC
35,ICICIPRULI,790.00,-20.88,42.0,H-MC,1.97,136142.0,354.0,42395.0,0.02,0.26,31.14,31.48,107.0,0.01,1.04,12.41,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-0.94,57.0,H-MC,1.67,245435.0,-2296.0,77361.0,-0.13,-0.93,31.52,30.30,92.0,-0.03,1.88,12.24,X40,ATH,ELECTRICAL


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-15.83,39.0,H-LC,6.30,303082.0,-12760.0,125415.0,-0.16,-4.04,41.38,35.67,5.0,-0.10,2.32,9.25,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.31,44.0,H-LC,13.63,218295.0,889.0,52609.0,0.58,0.41,24.10,24.61,48.0,0.02,1.67,55.45,XY24,BTT,FINANCE
31,HCLTECH,1943.91,0.65,47.0,H-LC,10.17,229644.0,-12280.0,75553.0,-0.08,-5.08,32.90,26.15,8.0,-0.16,1.76,8.52,X40,ATH,IT
56,RELIANCE,1533.00,-14.04,48.0,H-LC,6.30,215530.0,5164.0,23622.0,0.33,2.45,10.96,13.68,37.0,0.22,1.65,19.38,XY25,NTT,REFINERIES
80,UNITDSPR,1644.00,-13.80,48.0,H-LC,8.15,173382.0,-8377.0,43623.0,0.77,-4.61,25.16,19.39,86.0,-0.19,1.33,3.16,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-4.34,56.0,H-LC,1.27,237240.0,-8911.0,88111.0,-0.66,-3.62,37.14,32.18,16.0,-0.10,1.82,31.12,X200,ATH,IT
50,NESTLEIND,1377.00,-7.33,62.0,H-LC,2.89,285059.0,19633.0,37172.0,0.06,7.40,13.04,21.40,11.0,0.53,2.18,14.53,XY25,NTT,FMCG
41,ITC,452.00,-37.08,56.0,H-LC,3.02,202690.0,2552.0,18343.0,0.35,1.28,9.05,10.44,4.0,0.14,1.55,6.91,X40,NTT,FMCG
20,CIPLA,1795.00,-19.44,54.0,H-LC,5.97,214775.0,10275.0,31142.0,0.25,5.02,14.50,20.25,10.0,0.33,1.65,14.20,X40N,BTT,PHARMA
83,VBL,671.64,-15.83,39.0,H-LC,6.30,303082.0,-12760.0,125415.0,-0.16,-4.04,41.38,35.67,5.0,-0.10,2.32,9.25,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RAJOOENG,143.33,-32.03,49.0,H-SC,10.60,103100.0,600.0,40230.0,3.87,0.59,39.02,39.83,114.0,0.01,0.79,7.77,AR,ATH,MISC
65,SIEMENS,7969.85,-0.26,56.0,H-LC,29.14,160350.0,-25745.0,238136.0,2.05,-13.83,148.51,114.13,15.0,-0.11,1.23,18.08,AR,ATH,ELECTRICAL
27,GREENPANEL,537.00,203.62,55.0,M-SC,6.89,135493.0,-47585.0,126564.0,1.78,-25.99,93.41,43.14,230.0,-0.38,1.04,27.45,XY24,NTT,MISC
79,UNIONBANK,163.00,-16.28,56.0,M-LC,8.73,152517.0,11677.0,32806.0,1.23,8.29,21.51,31.59,66.0,0.36,1.17,36.98,XY24,NTT,BANKS
25,DEN,75.00,107.25,52.0,M-SC,3.30,58352.0,-26112.0,61649.0,1.22,-30.91,105.65,42.07,237.0,-0.42,0.45,23.92,AR,NTT,ENTERTAINMENT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VALIANTORG,838.0,-326.45,55.0,H-SC,8.42,96200.0,-37086.0,121683.0,-1.95,-27.82,126.49,63.47,139.0,-0.30,0.74,61.26,XR,NTT,CHEMICALS
22,COFFEEDAY,80.0,-39.64,56.0,L-SC,43.36,87955.0,-25594.0,65641.0,-1.80,-22.54,74.63,35.27,268.0,-0.39,0.67,115.27,XR,NTT,HOTELS
78,UJJIVANSFB,60.0,103.03,66.0,H-SC,12.71,128520.0,-13959.0,33479.0,-1.55,-9.80,26.05,13.70,163.0,-0.42,0.98,51.69,OX40N,NTT,BANKS
39,INFY,2275.0,-17.09,55.0,H-LC,9.78,323249.0,10361.0,161334.0,-0.98,3.31,49.91,54.87,3.0,0.06,2.48,10.18,X40,BTT,IT
76,TRIDENT,48.0,-8.30,54.0,M-SC,5.88,71334.0,-20920.0,47223.0,-0.96,-22.68,66.20,28.51,224.0,-0.44,0.55,22.06,XR,NTT,TEXTILES


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,103.03,66.0,H-SC,12.71,128520.0,-13959.0,33479.0,-1.55,-9.80,26.05,13.70,163.0,-0.42,0.98,51.69,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,106.19,63.0,M-SC,5.26,88818.0,-11959.0,12053.0,0.21,-11.87,13.57,0.09,245.0,-0.99,0.68,16.69,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.79,58.0,H-SC,2.27,222228.0,-47439.0,83780.0,1.07,-17.59,37.70,13.47,138.0,-0.57,1.70,12.40,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,114.84,50.0,M-SC,2.76,140864.0,-33695.0,33723.0,0.39,-19.30,23.94,0.02,221.0,-1.00,1.08,22.53,OX40N,NTT,PAINTS
19,CERA,9475.0,-20.05,46.0,H-SC,2.23,114156.0,-29647.0,56393.0,-0.50,-20.62,49.40,18.60,149.0,-0.53,0.87,25.97,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.44,53.0,H-LC,13.70,293543.0,-52413.0,176273.0,0.41,-15.15,60.05,35.80,1.0,-0.30,2.25,3.99,X40,BTT,IT
39,INFY,2275.00,-17.09,55.0,H-LC,9.78,323249.0,10361.0,161334.0,-0.98,3.31,49.91,54.87,3.0,0.06,2.48,10.18,X40,BTT,IT
41,ITC,452.00,-37.08,56.0,H-LC,3.02,202690.0,2552.0,18343.0,0.35,1.28,9.05,10.44,4.0,0.14,1.55,6.91,X40,NTT,FMCG
83,VBL,671.64,-15.83,39.0,H-LC,6.30,303082.0,-12760.0,125415.0,-0.16,-4.04,41.38,35.67,5.0,-0.10,2.32,9.25,X40N,ATH,FMCG
1,ABB,7934.00,-40.25,52.0,H-LC,12.29,249725.0,-11894.0,131106.0,0.79,-4.55,52.50,45.57,7.0,-0.09,1.91,6.26,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,2326.38,9.32,53.0,H-MC,3.74,188740.0,24070.0,43901.0,-0.12,14.62,23.26,41.28,89.0,0.55,1.45,23.77,X40N,ATH,AC
4,ANGELONE,3033.00,8.16,40.0,H-SC,6.40,191601.0,595.0,57097.0,0.03,0.31,29.80,30.21,157.0,0.01,1.47,21.26,X40N,NTT,FINANCE
84,VOLTAS,1918.49,-0.66,54.0,H-MC,3.17,208455.0,16713.0,79317.0,-0.37,8.72,38.05,50.08,99.0,0.21,1.60,16.11,XY25,ATH,AC
24,DABUR,735.00,-2.22,62.0,H-MC,6.02,212901.0,15873.0,73749.0,-0.29,8.06,34.64,45.49,102.0,0.22,1.63,22.15,XY24,BTT,FMCG
18,CAMS,5211.76,-3.83,55.0,H-SC,4.74,109410.0,7406.0,36521.0,0.42,7.26,33.38,43.06,122.0,0.20,0.84,25.62,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-43.66,68.0,H-MC,3.38,111761.0,11257.0,19379.0,0.83,11.20,17.34,30.49,79.0,0.58,0.86,47.24,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-39.02,64.0,M-SC,1.82,102990.0,11492.0,67263.0,0.25,12.56,65.31,86.07,223.0,0.17,0.79,49.56,XR,NTT,DURABLES
36,INDIAMART,4850.92,-50.53,54.0,H-SC,12.16,138023.0,14687.0,119072.0,1.08,11.91,86.27,108.45,119.0,0.12,1.06,36.92,AR,ATH,MISC
86,WIPRO,420.00,-10.16,57.0,M-LC,6.26,157474.0,6529.0,102925.0,-0.69,4.33,65.36,72.51,53.0,0.06,1.21,10.61,XR,NTT,IT
12,BANKINDIA,190.00,-29.19,64.0,H-MC,5.51,181930.0,2122.0,110286.0,0.68,1.18,60.62,62.52,88.0,0.02,1.39,35.67,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-171.59,65.0,M-SC,3.00,82638.0,-19572.0,112958.0,0.72,-19.15,136.69,91.37,208.0,-0.17,0.63,34.15,XY25,NTT,FINANCE
22,COFFEEDAY,80.00,-39.64,56.0,L-SC,43.36,87955.0,-25594.0,65641.0,-1.80,-22.54,74.63,35.27,268.0,-0.39,0.67,115.27,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.51,46.0,H-SC,14.33,88894.0,-12069.0,106602.0,-0.07,-11.95,119.92,93.63,148.0,-0.11,0.68,32.41,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.89,56.0,H-SC,13.06,94620.0,-3360.0,24961.0,0.32,-3.43,26.38,22.05,152.0,-0.13,0.72,37.54,XR,ATH,FINANCE
82,VALIANTORG,838.00,-326.45,55.0,H-SC,8.42,96200.0,-37086.0,121683.0,-1.95,-27.82,126.49,63.47,139.0,-0.30,0.74,61.26,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-39.64,56.0,L-SC,43.36,87955.0,-25594.0,65641.0,-1.80,-22.54,74.63,35.27,268.0,-0.39,0.67,115.27,XR,NTT,HOTELS
60,SAMMAANCAP,326.0,-171.59,65.0,M-SC,3.00,82638.0,-19572.0,112958.0,0.72,-19.15,136.69,91.37,208.0,-0.17,0.63,34.15,XY25,NTT,FINANCE
8,ATULAUTO,844.0,3667.57,61.0,M-SC,7.44,117504.0,-29864.0,81677.0,0.29,-20.26,69.51,35.16,236.0,-0.37,0.90,20.02,XY24,NTT,AUTO
75,TITAGARH,1548.0,-3.20,64.0,H-SC,6.49,158550.0,-33465.0,112349.0,-0.31,-17.43,70.86,41.08,158.0,-0.30,1.21,34.35,XY24,NTT,ENGINEERING
64,SHALBY,327.0,1105.74,77.0,M-SC,1.92,164061.0,-17082.0,62556.0,1.12,-9.43,38.13,25.10,235.0,-0.27,1.26,31.48,XY24,NTT,HEALTHCARE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.30
1,25,44.38
2,50,75.14


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.88
LC    35.45
MC    23.70
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.72
X40      16.05
X40N     13.13
XY25     10.27
XR        9.50
AR        8.43
OX40N     7.73
X200      1.82
MH        1.80
X5K       1.46
SR        1.12
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.85
H-SC    25.01
H-MC    20.62
M-SC    14.23
M-LC     7.49
M-MC     2.75
L-SC     1.64
L-LC     1.11
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,12.95,-15.44,73.88
FMCG,12.27,-3.53,38.62
FINANCE,9.07,-18.06,60.52
BANKS,7.79,-11.90,59.41
PAINTS,6.16,-14.93,40.65
MISC,6.11,-15.49,81.69
ELECTRICAL,5.86,-7.60,66.90
AC,3.80,2.13,31.02
AUTO,3.48,-15.63,67.77


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2718172.0,22
AR,1171415.0,9
XR,1053576.0,13
X40,872485.0,10
X40N,563150.0,10
OX40N,448199.0,10
XY25,409556.0,7
SR,197352.0,2
X5K,127675.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2759400.0,28
M-SC,1391562.0,17
H-LC,1360660.0,15
H-MC,1153206.0,15
M-LC,402579.0,5
M-MC,318333.0,2
L-SC,246330.0,3
L-MC,59682.0,1
L-LC,39501.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,912814.0,6
M-SC,XY24,810496.0,7
H-SC,AR,643233.0,5
H-LC,X40,581442.0,5
H-SC,XR,504349.0,6
H-MC,XY24,462332.0,4
H-LC,AR,369242.0,2
M-MC,XY24,318333.0,2
H-SC,X40N,246281.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 44.0 seconds
